In [1]:
import numpy as np
import math
import glob
import utils

from parse import load_ps
from pprint import pprint
from scipy.signal import medfilt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import DistanceMetric
from sklearn.metrics import classification_report

In [2]:
files = utils.files_in_order('poses_compressed/bicep')
X_train_names, X_test_names = train_test_split(files[:16], test_size=0.4, random_state=42)
y_train = utils.get_labels(X_train_names)
y_test = utils.get_labels(X_test_names)

pprint(X_train_names)
pprint(y_train)
pprint(X_test_names)
pprint(y_test)

16
array(['bicep_bad_10.npy', 'bicep_bad_3.npy', 'bicep_good_6.npy',
       'bicep_bad_5.npy', 'bicep_bad_8.npy', 'bicep_good_1.npy',
       'bicep_good_3.npy', 'bicep_bad_4.npy', 'bicep_bad_7.npy'],
      dtype='<U16')
array([0, 0, 1, 0, 0, 1, 1, 0, 0])
array(['bicep_bad_1.npy', 'bicep_bad_2.npy', 'bicep_bad_6.npy',
       'bicep_good_5.npy', 'bicep_good_4.npy', 'bicep_good_2.npy',
       'bicep_bad_9.npy'], dtype='<U16')
array([0, 0, 0, 1, 1, 1, 0])


In [4]:
def cal_vector_change(array) : 
    result = np.array([])
    times = array.shape[0]
    
    x_array = array[:,:,0]
    y_array = array[:,:,1]
    
    for t in range(1, times) :
        tmp = []
        v = np.sum(np.square(x_array[t-1] - x_array[t]))
        w = np.sum(np.square(y_array[t-1] - y_array[t]))

        result = np.append(result, np.sqrt(v+w))
 
    #print(result.shape)
    return result

In [22]:
def calculate_change_keypoints(names, flag):
    PART_NAMES = ['neck',  'rshoulder', 'relbow', 'rwrist', 'rhip', 'rknee', 'rankle',
                 'lshoulder', 'lelbow', 'lwrist', 'lhip', 'lknee', 'lankle']
    
    result = np.array([])
    for filename in names:
        output = []
        if flag == 0: # flag 0 이면 기존 training, test 
            ps = load_ps('poses_compressed/bicep/'+filename)
        else: # 1 flag 이면 새로 만든 값
            ps = load_ps('numpy/'+filename)
        
        poses = ps.poses
        
        for part in PART_NAMES :
            tmp = []
            for pose in poses : 
                tmp.append([getattr(pose, part).x, getattr(pose, part).y])

            output.append(tmp)
        
        output = np.array(output)
        output = output.transpose(1, 0, 2)    
        output = cal_vector_change(np.array(output))
    
    result = np.append(result, output)
    return result.reshape(len(names), -1)   
#calculate_change_keypoints(X_train_names, 0)

In [23]:
def KNN_with_keypoints(X_trains, X_test):
    predictions = []
    train_d_vectors = calculate_change_keypoints(X_train_names, 0)
    test_d_vector   = calculate_change_keypoints([X_test], 1)
    
    good, bad = [[] for i in range(2)]

    for idx, train_d_vector in enumerate(train_d_vectors): # X_test_names
        # Store the average distance to good and bad training examples
        test_d_vector = test_d_vector.reshape(-1, 1)
        
        dist = utils.DTWDistance(train_d_vector, test_d_vector)
        
        if y_train[idx]:
            good.append(dist)
        else:
            bad.append(dist)
             
    good_score = np.mean(good)
    bad_score  = np.mean(bad)

    if good_score < bad_score:
        return 1
    else:
        return 0
    
X_test_names = ['test.npy']
for X_test_name in X_test_names :
    print(KNN_with_keypoints(X_test_names, X_test_name)) # 기존 test data 

1
